***
![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


   # **Cortex Game: Reto SAS**
***

## **Connect to SASPy**

In [1]:
import saspy
sas_session = saspy.SASsession()
sas_session
;

Using SAS Config named: oda
Error trying to read authinfo file:C:\Users\JPY20\_authinfo
[Errno 2] No such file or directory: 'C:\\Users\\JPY20\\_authinfo'
Did not find key oda in authinfo file:C:\Users\JPY20\_authinfo

Please enter the OMR user id: a00829598@tec.mx
Please enter the password for OMR user : ········
SAS Connection established. Subprocess id is 7932



''

***
## Connect to Cortex Data Sets

In [2]:
%%SAS sas_session
libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

## Transform cloud SAS dataset to Python dataframe (pandas)


> **For reference**:

> 1. [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2. [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)


In [3]:
import pandas as pd
import numpy as np

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

# Merge_Data

In [4]:
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
755407,2755408.0,DAVIS,NICOLE,1.0,41.0,6100.0,High School,Rural,9.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
550464,2550465.0,WALLACE,RONALD,0.0,20.0,6000.0,University / College,Suburban,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


## Treat Missing Values

> Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.


In [5]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)

data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)

data_merge = data_merge.loc[data_merge['TotalGift'] < 8000]


print(data_merge.shape)

print(data_merge.isnull().sum())

data_merge.sample(3)

(999914, 22)
ID               0
LastName        23
FirstName        4
Woman            0
Age              0
Salary           0
Education        0
City             0
SeniorList       0
NbActivities     0
Referrals        0
Recency          0
Frequency        0
Seniority        0
TotalGift        0
MinGift          0
MaxGift          0
GaveLastYear     0
AmtLastYear      0
Contact          0
GaveThisYear     0
AmtThisYear      0
dtype: int64


,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
605219,2605220.0,SWARTZ,VICKIE,1.0,28.0,31500.0,2,1,1.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143565,2143566.0,SMITH,ROBERT,0.0,73.0,147700.0,1,2,7.0,2.0,...,2.0,7.0,550.0,150.0,400.0,0.0,0.0,0.0,0.0,0.0
655711,2655712.0,BEER,WANDA,1.0,34.0,34700.0,1,3,9.0,1.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Data Partition

In [6]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split

#data_merge = data_merge.sample(120000)
print(data_merge.shape)
train, validation = train_test_split(data_merge, test_size = 0.4, random_state=12345) 

#train.head()
train.sample(2)

(999914, 22)


,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
685057,2685058.0,BRAY,ELIZABETH,1.0,57.0,151700.0,2,1,6.0,3.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
343799,2343800.0,MARSH,SHERRILL,1.0,65.0,18800.0,1,1,4.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Models

## Linear Regresion

In [7]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

X_train = train[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear','City','Education', 'Contact']] 
Y_train = train['AmtThisYear']
X_valid = validation[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear','City','Education', 'Contact']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()
regr.fit(X_train,Y_train)
regr_predict=regr.predict(X_valid)

In [8]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

print(regr.score(X_train,Y_train))

15.279425624192177
8831.624384666764
93.9767225682337
0.00881693497546665


### **XG BOOST**

In [28]:
import xgboost as xgb

In [36]:
X_train = train[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear', 'Contact']] 
Y_train = train['AmtThisYear']

X_valid = validation[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear', 'Contact']] 
Y_valid = validation['AmtThisYear']

regr = xgb.XGBRegressor(booster = 'dart',tree_method = 'approx',n_estimators=170, max_depth=3, eta=0.3,subsample=0.75,colsample_bytree=0.8)
regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)


15.145897789901197
8840.996317583831
94.0265724015495


In [ ]:
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

***
# Round 2

In [37]:
import pandas as pd

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [38]:
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
#data_merge.head()
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
977187,2977188.0,PIERRE,JOYCE,1.0,67.0,62500.0,High School,City,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0
171428,2171429.0,CANCEL,CARLTON,0.0,63.0,232300.0,High School,Downtown,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


## Treating Missing Values

Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [39]:
data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)

data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)

data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)

data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)

data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)

data_merge.sample(3)

ID               0
LastName        23
FirstName        4
Woman            0
Age              0
Salary           0
Education        0
City             0
SeniorList       0
NbActivities     0
Referrals        0
Recency          0
Frequency        0
Seniority        0
TotalGift        0
MinGift          0
MaxGift          0
GaveLastYear     0
AmtLastYear      0
Contact          0
GaveThisYear     0
AmtThisYear      0
Age2             0
Pct              0
dtype: int64


,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear,Age2,Pct
863434,2863435.0,BOERIO,SHARICE,1.0,81.0,28300.0,1,2,5.0,2.0,...,120.0,20.0,70.0,0.0,0.0,0.0,0.0,0.0,0,1.004240
537102,2537103.0,MOORE,KEVIN,0.0,42.0,163900.0,1,2,5.0,1.0,...,30.0,30.0,30.0,1.0,25.0,0.0,0.0,0.0,1,1.000183
848014,2848015.0,GUNDERSON,MICHAEL,0.0,56.0,15500.0,1,2,5.0,3.0,...,40.0,40.0,40.0,0.0,0.0,1.0,0.0,0.0,0,1.002581


## Decision Tree Model

In [40]:

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.4, random_state=12345) 

train.head()

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear,Age2,Pct
634489,2634490.0,NAVARRO,HANNA,1.0,35.0,31700.0,1,4,3.0,2.0,...,20.0,20.0,20.0,1.0,25.0,0.0,0.0,0.0,1,1.000631
921403,2921404.0,WRIGHT,PAMELA,1.0,30.0,137600.0,1,4,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.000000


In [41]:
from sklearn.tree import DecisionTreeClassifier

X_train = train[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear','City','Education', 'Contact']] 
Y_train = train['AmtThisYear']
X_valid = validation[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear','City','Education', 'Contact']] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeClassifier(max_depth=5,criterion="entropy").fit(X_train,Y_train)
DT_predict_proba = DT_model.predict_proba(X_valid) #Predictions on Testing data
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
# Probabilities for each class
DT_probs = DT_model.predict_proba(X_valid)[:, 1]
print(DT_probs)

[0.01676458 0.02467057 0.05529132 ... 0.01572786 0.0890937  0.01046711]


In [42]:
#you can change the criteria
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix


confusion_matrix = confusion_matrix(Y_valid, DT_predict)
print(confusion_matrix)


[[340127      0      0 ...      0      0      0]
 [  9264      0      0 ...      0      0      0]
 [  1020      0      0 ...      0      0      0]
 ...
 [     7      0      0 ...      0      0      0]
 [     3      0      0 ...      0      0      0]
 [     9      0      0 ...      0      0      0]]


In [43]:
from sklearn.metrics import classification_report
print(classification_report(Y_valid, DT_predict))

C:\Users\JPY20\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.85      1.00      0.92    340213
        10.0       0.00      0.00      0.00      9310
        15.0       0.00      0.00      0.00      1023
        20.0       0.24      0.01      0.01     17143
        25.0       0.00      0.00      0.00      6511
        30.0       0.00      0.00      0.00      5730
        40.0       0.00      0.00      0.00      4673
        50.0       0.00      0.00      0.00      4675
        60.0       0.00      0.00      0.00       315
        70.0       0.00      0.00      0.00       275
        75.0       0.00      0.00      0.00      2521
        80.0       0.00      0.00      0.00       272
        90.0       0.00      0.00      0.00       304
       100.0       0.00      0.00      0.00      2412
       120.0       0.00      0.00      0.00       236
       125.0       0.00      0.00      0.00       212
       150.0       0.00      0.00      0.00       983
       175.0       0.00    

C:\Users\JPY20\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\JPY20\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Scoring New Data

### Prepare data for scoring

In [44]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='SCORE_RD2_NOCONTACT',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the Decision Tree model performed better than the Logistic Regression model based on the AUC criterion.

### Predict 'probability of giving' for members who were contacted

In [45]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

data_merge = scoring_data_contact

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge = data_merge.loc[data_merge['TotalGift'] < 9000]
print(data_merge.shape)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)


scoring_data_contact = data_merge

#scoring_data_contact.head()

X = scoring_data_contact[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear','City','Education', 'Contact']] 
DT_predict_contact=DT_model.predict_proba(X)[:,1]
scoring_data_contact['Prediction_prob'] = DT_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction_prob']]
scoring_data_contact = scoring_data_contact.rename({'Prediction_prob': 'ProbContact'}, axis=1) 
scoring_data_contact.head()

(999904, 22)


C:\Users\JPY20\AppData\Local\Temp\ipykernel_18696\2555421322.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
C:\Users\JPY20\AppData\Local\Temp\ipykernel_18696\2555421322.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],


,ID,ProbContact
0,2000001.0,0.050725
1,2000002.0,0.089094
2,2000003.0,0.089094
3,2000004.0,0.049319
4,2000005.0,0.089094


### Predict 'probability of giving' for members who were not contacted

In [46]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

data_merge = scoring_data_nocontact

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge = data_merge.loc[data_merge['TotalGift'] < 9000]
print(data_merge.shape)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)


scoring_data_nocontact = data_merge
#scoring_data_contact.head()

X = scoring_data_nocontact[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear','City','Education', 'Contact']] 
DT_predict_nocontact=DT_model.predict_proba(X)[:,1]
scoring_data_nocontact['Prediction_prob'] = DT_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction_prob']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction_prob': 'ProbNoContact'}, axis=1) 
scoring_data_nocontact.head()

(999904, 22)


C:\Users\JPY20\AppData\Local\Temp\ipykernel_18696\3117284037.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
C:\Users\JPY20\AppData\Local\Temp\ipykernel_18696\3117284037.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],


,ID,ProbNoContact
0,2000001.0,0.016765
1,2000002.0,0.010467
2,2000003.0,0.010467
3,2000004.0,0.010467
4,2000005.0,0.010467


In [47]:
result_Prob = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Prob.sort_values(by=['ID'], inplace=True)
result_Prob.sample(10)

,ID,ProbContact,ProbNoContact
413639,2413679.0,0.089094,0.034145
118888,2118900.0,0.050725,0.015728
86621,2086626.0,0.021129,0.015728
578323,2578380.0,0.089094,0.016765
787352,2787430.0,0.049319,0.054297
733649,2733722.0,0.049319,0.016765
652741,2652803.0,0.120904,0.015728
497494,2497542.0,0.049319,0.010467
806541,2806623.0,0.050725,0.016765
667031,2667096.0,0.089094,0.015728


## Exporting Results to a CSV File

In [48]:
result_Prob.to_csv('Round2_Output_prob.csv', index=False)

# Round 3

## Scoring New Data

### Prepare data for scoring

In [49]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

### Predict 'amount given' for members who were contacted

In [51]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

data_merge = scoring_data_contact

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge = data_merge.loc[data_merge['TotalGift'] < 9000]
print(data_merge.shape)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)


scoring_data_contact = data_merge
#scoring_data_contact.head()

X = scoring_data_contact[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear', 'Contact']]  

regr_predict_contact=regr.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

(999904, 22)


C:\Users\JPY20\AppData\Local\Temp\ipykernel_18696\3680811841.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
C:\Users\JPY20\AppData\Local\Temp\ipykernel_18696\3680811841.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],


,ID,AmtContact
0,2000001.0,63.578304
1,2000002.0,41.611595
2,2000003.0,41.567825
3,2000004.0,9.892123
4,2000005.0,39.721947


### Predict 'amount given' for members who were not contacted

In [52]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

data_merge = scoring_data_nocontact

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=10)
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=10)

data_merge['Age2'] = np.where(data_merge['Age']<= 46, 1, 0)
data_merge['Pct'] = round(((data_merge['TotalGift']/data_merge['Salary'])+1),10)
data_merge.replace([np.inf,-np.inf],np.nan, inplace = True)
data_merge[['Pct']] = data_merge[['Pct']].fillna(value=0)

data_merge = data_merge.loc[data_merge['TotalGift'] < 9000]
print(data_merge.shape)

data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
                        [1,2,3], inplace=True)
data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],
                        [1,2,3,4], inplace=True)


scoring_data_nocontact = data_merge

#scoring_data_nocontact.head()

X = scoring_data_nocontact[['NbActivities','Frequency', 'Seniority','SeniorList','Age','Woman','Salary','GaveLastYear','AmtLastYear', 'Contact']]  

regr_predict_nocontact=regr.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

(999904, 22)


C:\Users\JPY20\AppData\Local\Temp\ipykernel_18696\1841915893.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['Education'].replace(['University / College', 'High School', 'Elementary'],
C:\Users\JPY20\AppData\Local\Temp\ipykernel_18696\1841915893.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merge['City'].replace(['Downtown','City', 'Suburban', 'Rural'],


,ID,AmtNoContact
0,2000001.0,19.232441
1,2000002.0,8.525219
2,2000003.0,7.905926
3,2000004.0,3.108669
4,2000005.0,5.305988


In [53]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,63.578304,19.232441
1,2000002.0,41.611595,8.525219
2,2000003.0,41.567825,7.905926


In [54]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

## Exporting Results to a CSV File

In [55]:
import pandas as pd

NB = 150000

probabilities = pd.read_csv("Round2_Output_prob.csv")
amounts = pd.read_csv("Round2_Output_amt.csv")

def Calc_Uplift(raw_data):
    return ((raw_data['AmtContact']*raw_data['ProbContact']) - (raw_data['AmtNoContact']*raw_data['ProbNoContact']))

raw_submission = pd.merge(probabilities, amounts, on=["ID"], how="right")
raw_submission["Uplift"] = raw_submission.apply(lambda row: Calc_Uplift(row), axis=1)

# Sorting data by descending Uplift value
raw_submission.sort_values(by=['Uplift'], ascending=False, inplace=True)


In [56]:
# Export the final csv file

submission = raw_submission.head(NB)

submission.to_csv('Round2 Output final.csv', index=False)
